# Basic Filtering that I have been using for analysis so far

It's not very technical, open to improvements

In [1]:
import pandas as pd

In [2]:
#read csv files and store them as pandas dataframes

raw_articles_df = pd.read_csv ('articles.csv')
raw_train_df = pd.read_csv ('transactions_train.csv')
raw_customers_df = pd.read_csv ('customers.csv')

In [3]:
#take sample of transaction data (10% of 32 million individual transactions)

df_sample = raw_train_df.sample(frac=.1, replace=True, random_state=1)
pd.to_datetime(df_sample['t_dat']) 

12710949   2019-06-26
21463275   2020-01-20
6762380    2019-02-27
12325960   2019-06-22
491263     2018-09-29
              ...    
9846753    2019-05-05
5251936    2019-01-21
29469699   2020-07-25
5647044    2019-01-31
14734211   2019-07-29
Name: t_dat, Length: 3178832, dtype: datetime64[ns]

In [12]:
#example code for adding a column from one df to another
#can add patterns, department, basically anything. this just adds product type, age, and product color

id_type_dict = dict(zip(raw_articles_df.article_id, raw_articles_df.product_type_name))
df_sample['product_type'] = df_sample['article_id'].map(id_type_dict)

id_color_dict = dict(zip(raw_articles_df.article_id, raw_articles_df.colour_group_name))
df_sample['color'] = df_sample['article_id'].map(id_color_dict)

id_age_dict = dict(zip(raw_customers_df.customer_id, raw_customers_df.age))
df_sample['age'] = df_sample['customer_id'].map(id_age_dict)

In [15]:
df = df_sample.set_index('customer_id')

#extract month from date if you want
df['month'] = pd.DatetimeIndex(df['t_dat']).month

df['age_bin'] = pd.cut(df['age'], [0, 20, 30, 40, 50, 60, 70, 80, 90, 100], 
                              labels=['0-20', '20-30', '30-40', '40-50','50-60','60-70','70-80', '80-90','90-100'])

#drop whatever columns you don't want, for example
df = df.drop(['price', 'sales_channel_id', 'article_id', 't_dat', 'age'], axis = 1)

In [16]:
#drop highest colors and category, and low sellers

df = df[df.color != 'Black' ]
df = df[df.color != 'White' ]
df = df[df.color != 'Dark Blue' ]

df = df[df.product_type != 'Trousers' ]

df = df[df.groupby('product_type').product_type.transform(len) > 1000]
df = df.dropna()

df.head()

,product_type,color,month,age_bin
customer_id,,,,
505d66fbc88b4300e6b4d11c26bfe4dd3857e21c07f86aa4e5d293b84201f619,Vest top,Orange,6,20-30
016fb4c5d61afcb3ff1b907c0d6f00ab9f4e4071ca22a4dad7d88e410ae35fda,Dress,Yellowish Brown,6,70-80
88beadfe55a64381384b9c83334e3bb13be26374873b0dc108343f6bf3e7e9b9,T-shirt,Red,9,20-30
9c8761fcb2106e9664601a1cd87b79bc99b9049b0dbe787eb1efeb178fa11c81,T-shirt,Beige,7,30-40
78971a184035137737aecedfce711be93b6a603161d908a33313186bdbc78303,Cardigan,Light Beige,3,50-60
